In [ ]:
#import nbpresent
#nbpresent.__version__


In [ ]:
import time

import bokeh.plotting
bokeh.plotting.output_notebook()
now=time.gmtime()
print("This notebook was last updated: "+time.asctime(now)+" UTC")

Let's compare catalogs

In [ ]:
import astropy.io.votable as votab
import astropy.io.ascii 
import astropy.units as u

import numpy as np

#download latest target lists:

latest_RV_url = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/PlanetPopulation/RVplanets_ipac_2016-05-15.votable"
exoarch_confirmed_url = "http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=votable&select=*"
exoplanets_eu_url = "http://www.exoplanet.eu/catalog/votable/" 
EXOSIM_stars = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/StarCatalog/mission_exocat.votable"
oec_latest_tab = "https://raw.githubusercontent.com/douglase/oec_tables/master/comma_separated/open_exoplanet_catalogue.txt"


In [ ]:
#parse
exocat = votab.parse(EXOSIM_stars,).get_first_table().to_table()
#knownRV = votab.parse("../EXOSIMS/EXOSIMS/PlanetPopulation/RVplanets_ipac_032216.votable").get_first_table().to_table()
knownRV = votab.parse(latest_RV_url).get_first_table().to_table()
#exoplanets_dot_eu = votab.parse(exoplanets_eu).get_first_table().to_table()
#create 

exoarch_confirmed_file=astropy.utils.data.download_file(exoarch_confirmed_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=10) #regular call was timing out
exoarchConfirmed = votab.parse(exoarch_confirmed_file).get_first_table().to_table()
last_download_time=time.asctime(now)

In [ ]:
if exoarchConfirmed["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    exoarchConfirmed["pl_orbper"].unit=u.day #fix units
if knownRV["pl_orbper"].unit == "days":
    print("days not a recognized unit, replacing with day")
    knownRV["pl_orbper"].unit=u.day #fix units


In [ ]:
'''exoplanets_eu_file = astropy.utils.data.download_file(exoplanets_eu_url, 
                                                        cache=False,
                                                        show_progress=True,
                                                        timeout=60) 
exoplanets_dot_eu = votab.parse(exoplanets_eu_file)#.get_first_table().to_table() #returns E05: Invalid boolean value 'Yes'
'''

In [ ]:
#Define open exoplanet catalog column names:
oec_cols=["name",#  1: Primary identifier of planet
          "binary",#  2: Binary flag [0=no known stellar binary companion; 1=P-type binary (circumbinary); 2=S-type binary; 3=orphan planet (no star)]
          "pl_massj",##  3: Planetary mass [Jupiter masses]
          "pl_radiusj",##  4: Radius [Jupiter radii]
          "pl_orbper",#  5: Period [days]
          "pl_orbsmax",#  6: Semi-major axis [Astronomical Units]
          "pl_orbeccen",#  7: Eccentricity
          "pl_orblper",#  8: Periastron [degree]
          "longitude", ##  9: Longitude [degree] https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/issues/292
          "ascendingnode",#"10: Ascending node [degree]",
          "pl_orbincl",#"# 11: Inclination [degree]
          "pl_eqt",# 12: Surface or equilibrium temperature [K]
          "pl_age",# 13: Age [Gyr]
          "pl_discmethod",#,14: Discovery method
          "pl_disc",# 15:" Discovery year [yyyy]
          "rowupdate",# 16: Last updated [yy/mm/dd]
          "ra_str",# 17: Right ascension [hh mm ss]
          "dec_str",# 18: Declination [+/-dd mm ss]
          "st_dist",# 19: Distance from Sun [parsec]
          "st_mass",# 20: Host star mass [Solar masses]
          "st_rad",# 21: Host star radius [Solar radii]
          "st_metfe",# 22: Host star metallicity [log relative to solar]
          "st_teff",# 23: Host star temperature [K],"24: Host star age [Gyr]"]
          "st_age",# 24: Host star age [Gyr]
          "list",#detection status list
]
          
oec=astropy.io.ascii.read(oec_latest_tab,data_start=31,names=oec_cols,delimiter=",",guess=False)
oec=oec[oec["list"]=="Confirmed planets"]
oec["pl_orbper"].unit=u.day
oec["st_dist"].unit=u.parsec

In [ ]:
def scatter_with_labels(p,x,y,names,**kwargs):
    #data source for mouseover
    if isinstance(names[0],bytes):
        names= [j.decode('utf-8') for j in names]
    source = bokeh.models.ColumnDataSource(data=dict(
        x=x,
        y=y,
        name=names, #convert bytes to strings
    ))

    p.scatter(x,y,line_color=None,source=source,**kwargs)
tooltips=[("index", "$index"),
          ("(x,y)", "(@x, @y)"),
          ("name", "@name")]

In [ ]:

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"

p = bokeh.plotting.figure(tools=TOOLS, x_range=( 10000,0))

p.scatter(exocat['st_teff'], (exocat['st_mass']), radius=50,
          fill_color='gray', fill_alpha=0.5,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_mass"]), radius=100,
fill_color="orange", fill_alpha=0.2,legend="Known RV",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Mass [M_sun]"
bokeh.plotting.show(p)


In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1e-3,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'],exocat['st_lbol'],
                    exocat["hip_name"],
                    color="blue",legend="exocat",size=10,fill_alpha=0.1)

scatter_with_labels(p,knownRV["st_teff"],10**(knownRV["st_lum"]), 
                    knownRV["pl_name"],
                    color="orange",legend="Known RV",size=10,fill_alpha=.3)

p.title.text = "H-R Diagram"
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Luminousity [L_sun]"
bokeh.plotting.show(p)


In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))


p.scatter(exocat['st_teff'], (exocat['st_dist']), 
                              exocat["hip_name"],
          size=5,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]),
                              knownRV["pl_name"], size=10,
          fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), size=10,
fill_color="green", fill_alpha=0.1,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)




In [ ]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'], (exocat['st_dist']),
                    exocat["hip_name"],
                    radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",)
scatter_with_labels(p,knownRV['st_teff'], (knownRV["st_dist"]), knownRV["pl_name"],
                    radius=100,
                    fill_color="blue", fill_alpha=0.2,legend="Known RV",)
scatter_with_labels(p,exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), exoarchConfirmed["pl_hostname"],
                    radius=100,
                    fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [ ]:
hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(0, 25),  )

scatter_with_labels(p,exoarchConfirmed["st_optmag"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="green",legend="IPAC Exoplanet Archive",
                    size=12,fill_alpha=0.34)

scatter_with_labels(p,knownRV["st_optmag"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="Known RV")

#bokeh.plotting.show(p)
p.xaxis.axis_label = "Optical Mag"
p.yaxis.axis_label = "distance [pc]"

#what is optical mag? Johnson v_mag?




In [ ]:
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35))#x_range=(0, 10000),  )

scatter_with_labels(p,exoarchConfirmed["pl_orbper"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="gray",legend="IPAC Exoplanet Archive",size=6,fill_alpha=0.6)

scatter_with_labels(p,knownRV["pl_orbper"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="blue",legend="Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p,oec["pl_orbper"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p.xaxis.axis_label = "orbital period [d]"
p.yaxis.axis_label = "distance [pc]"



In [ ]:
hover = bokeh.models.HoverTool(tooltips=tooltips)

def sep_as(dist,period):
    '''
    calculate the seperation in arcseconds
    $p[years]^2=a[AU]^3$

    s["]=a[AU]/d[pc]

    $s["]=(p)^{2/3}/d$
    '''
    return (period.to(u.year).value**(2/3)/dist.to(u.parsec).value)

tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p2 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 45),x_range=(1e-2, 40),  )
#https://github.com/bokeh/bokeh/issues/3531

scatter_with_labels(p2,sep_as(exoarchConfirmed["st_dist"],exoarchConfirmed["pl_orbper"]),
                    exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    color="gray",legend="IPAC Exoplanet Archive",size=10,fill_alpha=0.6)

scatter_with_labels(p2,sep_as(knownRV["st_dist"],knownRV["pl_orbper"]),knownRV["st_dist"], 
                    knownRV["pl_hostname"] +knownRV["pl_letter"],
                    color="blue",legend="EXOSIMS Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p2,sep_as(oec["st_dist"],oec["pl_orbper"]),oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p2.xaxis.axis_label = "seperation [arcseconds]"
p2.yaxis.axis_label = "distance [pc]"


p2.title.text = "Seperation (from period, assuming circular orbits)"


bokeh.plotting.show(p2)

In [ ]:
bokeh.io.curdoc().clear()#https://github.com/bokeh/bokeh/issues/3943

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p1 = bokeh.plotting.Figure(tools=tools,x_axis_type="log",  y_range=(-5, 45),x_range=(1e-3, 1000),  )

scatter_with_labels(p1,exoarchConfirmed["pl_msinij"],exoarchConfirmed["st_dist"],
                     exoarchConfirmed["pl_hostname"] +exoarchConfirmed["pl_letter"],        
                    size=12,fill_alpha=0.6,
                    color="gray",legend="IPAC Exoplanet Archive")

scatter_with_labels(p1,knownRV["pl_msinij"],knownRV["st_dist"], 
                     knownRV["pl_hostname"] +knownRV["pl_letter"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="EXOSIMS Known RV")

scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed Planets",size=10,fill_alpha=.35)
p1.xaxis.axis_label = "orbital period [d]"
scatter_with_labels(p1,oec["pl_massj"],oec["st_dist"],oec["name"],
                    size=10,color='orange',fill_alpha=.3)
p1.xaxis.axis_label = "msini [Jupiter]"
p1.yaxis.axis_label = "distance [pc]"
p1.title.text = "Catalogs last queried: "+last_download_time

In [ ]:
grid = bokeh.layouts.gridplot([p2,p1], ncols=2, plot_width=450, plot_height=500)
bokeh.plotting.output_file("pc_msini_and_sep.html", title=last_download_time)
#bokeh.plotting.show(p)
bokeh.plotting.show(grid)